<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2

Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [1]:
using LinearAlgebra

In [2]:
using Printf

function print_matrix(M)
  for i in 1:size(M, 1)
    for j in 1:size(M, 2)
      @printf("%10.4f ", M[i, j])
    end
    println()
  end
end


print_matrix (generic function with 1 method)

## Questão 1

**Substituição regressiva**

In [3]:
function back_substitution(U, b)
  n = length(b)
  x = zeros(n)

  for i = n:-1:1
    x[i] = b[i]
    for j = i+1:n
      x[i] = x[i] - U[i,j] * x[j]
    end
    x[i] = x[i] / U[i,i]
  end

  return x
end

back_substitution (generic function with 1 method)

**Substituição direta**

In [4]:
function forward_substitution(L, b)
  n = length(b)
  x= zeros(n)

  for i=1:n
    x[i] = b[i]
    for j=1:i-1
      x[i] = x[i] - L[i,j]*x[j]
    end
    x[i] = x[i]/L[i,i]
  end

  return x
end

forward_substitution (generic function with 1 method)

In [5]:
n = 4
U = triu(rand(n, n))
L = tril(rand(n, n))
b = rand(n)

x_back = back_substitution(U, b)
x_forward = forward_substitution(L, b)

println("U = ")
print_matrix(U)
println("Solução (back-substitution):")
print_matrix(x_back)

println("\nL = ")
print_matrix(L)
println("Solução (forward-substitution):")
print_matrix(x_forward)


U = 
    0.2946     0.0319     0.0875     0.3126 
    0.0000     0.1845     0.9318     0.4884 
    0.0000     0.0000     0.8128     0.0300 
    0.0000     0.0000     0.0000     0.3367 
Solução (back-substitution):
    2.7104 
   -6.8197 
    1.1917 
    0.3961 

L = 
    0.7364     0.0000     0.0000     0.0000 
    0.6320     0.1888     0.0000     0.0000 
    0.6927     0.6363     0.4645     0.0000 
    0.4712     0.5287     0.7022     0.6613 
Solução (forward-substitution):
    1.0985 
   -3.4339 
    5.1770 
   -3.3322 


## Questão 2

**Fatoração LU (sem pivoteamento)**

In [6]:
function flu(A)
  n = size(A, 1)
  L = Matrix{Float64}(I, n, n)
  U = copy(A)

  for k in 1:n-1
    for i in k+1:n
      L[i, k] = U[i, k] / U[k, k]
      U[i, k:n] -= L[i, k] * U[k, k:n]
    end
  end

  return L, U
end


flu (generic function with 1 method)

**Fatoração de Cholesky**

In [7]:
function fchol(A)
  n = size(A, 1)
  L = zeros(n, n)

  for i in 1:n
    for j in 1:i
      sum_k = dot(L[i, 1:j-1], L[j, 1:j-1])
        if i == j
          L[i, j] = sqrt(A[i, i] - sum_k)
        else
          L[i, j] = (A[i, j] - sum_k) / L[j, j]
        end
    end
  end

  return L
end

fchol (generic function with 1 method)

In [8]:
function solve_system(n)
  A = diagm(0 => fill(4.0, n),
            1 => fill(-1.0, n-1), -1 => fill(-1.0, n-1),
            2 => fill(-1.0, n-2), -2 => fill(-1.0, n-2))
  b = A[:, n]

  println("\n============================")
  println("n = $n")

  t_lu = @elapsed begin
    L, U = flu(A)# explicar pq admite
    y = forward_substitution(L,b)
    x_lu = back_substitution(U,y)
  end

  t_chol = @elapsed begin
    L = fchol(A)# explicar pq admite
    y = forward_substitution(L,b)
    x_chol = back_substitution(L',y)
  end

  x_true = zeros(n)
  x_true[end] = 1 # por que??? explicar
  err_lu = norm(x_lu - x_true) / norm(x_true)
  err_chol = norm(x_chol - x_true) / norm(x_true)

  println("== Fatoração LU ==")
  println("Tempo: ", t_lu, " segundos")
  println("Erro relativo: ", err_lu)

  println("\n== Fatoração de Cholesky ==")
  println("Tempo: ", t_chol, " segundos")
  println("Erro relativo: ", err_chol)
end

solve_system (generic function with 1 method)

In [9]:
for n in [10, 100, 200, 400]
    solve_system(n)
end


n = 10
== Fatoração LU ==
Tempo: 6.6145e-5 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 2.3431e-5 segundos
Erro relativo: 2.4317759426983836e-16

n = 100
== Fatoração LU ==
Tempo: 0.010296549 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.003158518 segundos
Erro relativo: 8.274363007295436e-16

n = 200
== Fatoração LU ==
Tempo: 0.171408424 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.010449267 segundos
Erro relativo: 9.720503087223581e-16

n = 400
== Fatoração LU ==
Tempo: 0.389887224 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.111483929 segundos
Erro relativo: 1.3701608414352226e-15


## Questão 3

## Questão 4

Note que a matriz $A$ é simétrica e diagonalmente dominante com elementos positivos na diagonal. Logo, é simétrica positiva definida e admite fatoração de Cholesky.

In [13]:
n=6

A = Matrix{Float64}(I, n, n)
A[1,2:n] .= 0.1
A[2:n,1] .= 0.1

G = fchol(A)
println("Matriz G:")
print_matrix(G)

Matriz G:
    1.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.1000     0.9950     0.0000     0.0000     0.0000     0.0000 
    0.1000    -0.0101     0.9949     0.0000     0.0000     0.0000 
    0.1000    -0.0101    -0.0102     0.9949     0.0000     0.0000 
    0.1000    -0.0101    -0.0102    -0.0103     0.9948     0.0000 
    0.1000    -0.0101    -0.0102    -0.0103    -0.0104     0.9948 


Observe que a esparsidade de $A$ não é preservada em $G$ devido à natureza da fatoração de Cholesky. Isso porque os elementos da 1° linha "contaminam" as próximas linhas.

Para resolver esse problema, podemos encontrar uma matriz de permutação $P$ que transfere a primeira linha/coluna de $A$ para a última linha/coluna.

In [14]:
I6 = Matrix(I, n, n)
P = I6[[2, 3, 4, 5, 6, 1], :]

6×6 Matrix{Bool}:
 0  1  0  0  0  0
 0  0  1  0  0  0
 0  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1
 1  0  0  0  0  0

Observe $PAP^{-1}=PAP^T$

In [16]:
PAPt = P*A*P'

6×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.1
 0.0  1.0  0.0  0.0  0.0  0.1
 0.0  0.0  1.0  0.0  0.0  0.1
 0.0  0.0  0.0  1.0  0.0  0.1
 0.0  0.0  0.0  0.0  1.0  0.1
 0.1  0.1  0.1  0.1  0.1  1.0

Assim, calculando o seu fator de Cholesky, temos

In [17]:
G = fchol(PAPt)
println("Matriz G:")
print_matrix(G)

Matriz G:
    1.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.0000     1.0000     0.0000     0.0000     0.0000     0.0000 
    0.0000     0.0000     1.0000     0.0000     0.0000     0.0000 
    0.0000     0.0000     0.0000     1.0000     0.0000     0.0000 
    0.0000     0.0000     0.0000     0.0000     1.0000     0.0000 
    0.1000     0.1000     0.1000     0.1000     0.1000     0.9747 


Note que $G$ preserva a esparsidade da matriz.